In [2]:
import tweepy

In [10]:
import urllib3
import urllib3.request

import csv
import json
from pandas.io.json import json_normalize

import pandas as pd
pd.options.display.max_colwidth = 1000
import numpy as np
import re

import nbimporter
import utility_functions as uf

In [4]:
from importlib import reload
reload(uf)

Importing Jupyter notebook from utility_functions.ipynb


<module 'utility_functions' from 'utility_functions.ipynb'>

In [1]:
####input your credentials here
consumer_key = '4ha4rLgP6Ci6fEZtaqttGTKoA'
consumer_secret = '5ckLaCgfTdfmWM7qS9f2w05pDCSIWRCTHlm7RLnKwK9tCWIz9P'
access_token = '602145669-jHmxtsl0wSZDFeZxi81GcTzYrD87dRBhF78ip0qo'
access_token_secret = 'YFLMmVVdcN4gb4KDX3MeOjbjxoKnnsFvjKxjRGMkkEZ5D'

In [5]:
#define inputs here
input_hashtag = "#giletjaune"
language = "fr"
since_date = "2018-03-07"
#n_tweets = 1

In [47]:
uf.db_init()
rt = 0

In [ ]:
from twarc import Twarc
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret,tweet_mode= 'extended')
for tweet in t.search(input_hashtag, lang=language):
    with open('tweets_twarc/tweet'+str(tweet['id'])+'.json', 'w', encoding='utf8') as file:
        ####you need to create a 'tweets_twarc' folder
       
        json.dump(tweet, file)

rate limit exceeded: sleeping 614.4272046089172 secs


In [158]:
## this cell will be deprecated when switching to utility function
#databases initialisations
rt = 0
def db_init():
    global df_users
    global df_tweets
    global df_hashtags
    global df_users_mentions
    global df_retweet_users
    
    #initialisation of databases, df files

    d_users = {'user_id': [],
               'name':[],
               'screen_name':[],
               'location':[],
               'description':[],
               'url':[],
               'followers_count':[],
               'friends_count':[],
               'listed_count':[],
               'created_at':[],
               'favourites_count':[],
               'geo_enabled':[],
               'verified':[],
               'statuses_count':[],
               'lang':[],
               'contributors_enabled':[]
              }
    df_users = pd.DataFrame(data=d_users)

    d_tweets = {'tweet_id': [],
                'created_at': [],
                'text': [],
                'truncated': [],
                'source': [],
                'in_reply_to_status_id': [],
                'in_reply_to_user_id': [],
                'in_reply_to_screen_name': [],
                'user_id': [],
                'geo': [],
                'coordinates': [],
                'place': [],
                'contributors': [],
                'is_quote_status': [],
                'retweet_count': [],
                'favorite_count': [],
                'favorited': [],
                'retweeted': [],
                'lang': []         
               }
    df_tweets = pd.DataFrame(data=d_tweets)


    d_hashtags={'hashtag': [], 'tweet_id': []}
    df_hashtags = pd.DataFrame(data=d_hashtags)

    d_users_mentions = {'tweet_id':[], 'user_id': [], 'screen_name':[], 'name':[]}
    df_users_mentions = pd.DataFrame(data=d_users_mentions)

    d_retweet_users = {'user_screen_name':[], 'original_user_screen_name':[], 'original_tweet_id':[]}
    df_retweet_users= pd.DataFrame(data=d_retweet_users)


# store all hashtags related to a tweet
def store_hashtag(tweet):
    global df_hashtags
    for raw_hash in tweet['entities']['hashtags']:
            df_hashtags = df_hashtags.append({'hashtag': raw_hash['text'] ,'tweet_id':tweet['id_str']}, ignore_index=True)

#store all user mentions contained in the tweet
def store_user_mentions(tweet):
    global df_users_mentions
    tweet_id = tweet['id_str']
    for raw_mention in tweet['entities']['user_mentions']:
        screen_name = raw_mention['screen_name']
        name = raw_mention['name']
        user_id = raw_mention['id_str']
        df_users_mentions = df_users_mentions.append({'tweet_id':tweet_id, 'user_id': user_id, 'screen_name':screen_name, 'name':name},ignore_index=True)

#store the author of the tweet        
def store_user(tweet):
    global df_users
    user = tweet['user']
    if user['id_str'] not in df_users['user_id'].values:
        
        d_user = {'user_id': user['id_str'],
                  'name':user['name'],
                  'screen_name':user['screen_name'],
                  'location':user['location'],
                  'description':user['description'],
                  'url':user['url'],
                  'followers_count':user['followers_count'],
                  'friends_count':user['friends_count'],
                  'listed_count':user['listed_count'],
                  'created_at':user['created_at'],
                  'favourites_count':user['favourites_count'],
                  'geo_enabled':user['geo_enabled'],
                  'verified':user['verified'],
                  'statuses_count':user['statuses_count'],
                  'lang':user['lang'],
                  'contributors_enabled':user['contributors_enabled']
                 }
        df_users = df_users.append(d_user,ignore_index=True)

#store the content of the tweet
def store_tweet(tweet):
    global df_tweets
    text = re.sub(r"http\S+", "", tweet['full_text']) ## Removes URLs from full text
    df_tweets = df_tweets.append({'tweet_id':  tweet['id_str'],
            'created_at': tweet['created_at'],
            'text': text,
            'truncated': tweet['truncated'],
            'source': tweet['source'],
            'in_reply_to_status_id': tweet['in_reply_to_status_id_str'],
            'in_reply_to_user_id': tweet['in_reply_to_user_id_str'],
            'in_reply_to_screen_name': tweet['in_reply_to_screen_name'],
            'user_id': tweet['user']['id_str'],
            'geo': tweet['geo'],
            'coordinates': tweet['coordinates'],
            'place': tweet['place'],
            'contributors': tweet['contributors'],
            'is_quote_status': tweet['is_quote_status'],
            'retweet_count': tweet['retweet_count'],
            'favorite_count': tweet['favorite_count'],
            'favorited': tweet['favorited'],
            'retweeted': tweet['retweeted'],
            'lang': tweet['lang'] },ignore_index=True)

#called when the tweet is a retweet: store the original author of the tweet as well as the user who retweeted it,
# and the original id of the tweet
def store_retweet_user(tweet):
    global df_retweet_users
    if is_a_retweet(tweet):
        df_retweet_users = df_retweet_users.append({'user_screen_name':tweet['user']['screen_name'],
                                                    'original_user_screen_name':tweet['retweeted_status']['user']['screen_name'],
                                                    'original_tweet_id':tweet['retweeted_status']['id_str']}, ignore_index=True)
##Never used
##T0D0: proper encapsulation
def store_influent_users(n):
    #n is the number of influent users return
    df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).head()
    for i in range(n):
        d_influent_users = {'user_id':df_favorite_count[0], 
                            'name':[],
                            'retweet_count':[],
                            'favorite_count':[], 
                            'description':[] 
                           }
        
        
def is_a_retweet(tweet):
    #### returns a true if the tweet is a retweet, false if it's an original tweets 
    global rt
    rt += 1
    return tweet.get('retweeted_status',None) != None

def process_json_tweet(tweet):
    tweet = json.load(tweet)
    
    if is_a_retweet(tweet):
        store_retweet_user(tweet)
        
    else:  #store tweets only when they are original tweets
        store_tweet(tweet)
        store_user(tweet)
        store_hashtag(tweet)
        store_user_mentions(tweet)
    



In [159]:
db_init()

In [160]:
#### loading the databases
t=0
rt = 0
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur
import os

path_to_json = 'tweets_twarc/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for json_file in json_files:
    
    with open(path_to_json+json_file) as tweet:
        t +=1
        process_json_tweet(tweet)
        
        
        

In [12]:
df_tweets.head(10);

In [13]:
#tweet example
df_tweets['text'].iloc[0]

'[ #GiletsJaunes #ActeXVIII ] \n\n⚡️Plusieurs centaines de #GiletsJaunes défilent dans les rues de #Paris ❗️\n\n📢🗣«\xa0#Macron démission / #Castaner en prison !\xa0»\n\n#giletjaune #Acte18bis #EmmanuelMacron #acteXVIIIbis #manifestations #manifestation #Greve19mars #France #politique \n\n⬇️📹 '

In [55]:
d = pd.to_datetime(df_tweets['created_at'])
print(d.min())
print(d.max())
print(len(df_tweets.index)) 


2019-03-19 18:25:03+00:00
2019-04-07 11:40:44+00:00
3149
number of files scraped, including possible retweets: 23812
number of retweets 44475


In [15]:
#primary keys are ok if both return false
print(df_tweets.duplicated('tweet_id').any())
print(df_users.duplicated('user_id').any())

False
False


# Aggregations


## Top users

In [136]:
df_most_favorited_users = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_retweeted_users = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_favorited_users['favorite_count'] = df_most_favorited_users['favorite_count'].astype('int32')
df_most_retweeted_users['retweet_count'] = df_most_retweeted_users['retweet_count'].astype('int32')

df_most_favorited_users['user_id'] = df_most_favorited_users['user_id'].astype(object)
df_most_retweeted_users['user_id'] = df_most_retweeted_users['user_id'].astype(object)

In [137]:
df_most_favorited_users_with_info = df_most_favorited_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [138]:
df_most_retweeted_users_with_info.head(15);

In [139]:
df_most_favorited_users_with_info.head(15);

In [140]:
df_most_retweeted_users_with_info.reset_index()
df_most_favorited_users_with_info.reset_index();

In [141]:
print("Most Retweeted Users");
print(df_most_retweeted_users_with_info['screen_name']);
print("Most Favorited Users");
print(df_most_favorited_users_with_info['screen_name']);

Most Retweeted Users
user_id
1028668294751039488     GiletsJaunesFr
910127605277495297           LeMediaTV
784636611183214592        FranceNews24
435224364                    Callystor
1093171456308072448          CitateurF
1084126493788332033         AA36514431
1066350050069176321           MarchaiJ
338955445                    tcabarrus
1083279881209896966     LorentzMathias
840670437050875904             BBR4369
155606403                  LEDOUAISIEN
480752391                    jmAlric11
33962372                 ChLECHEVALIER
1062458079470600194    collCartonJaune
29707536                     quatremer
Name: screen_name, dtype: object
Most Favorited Users
user_id
1028668294751039488    GiletsJaunesFr
910127605277495297          LeMediaTV
338955445                   tcabarrus
1093171456308072448         CitateurF
29707536                    quatremer
784636611183214592       FranceNews24
1083279881209896966    LorentzMathias
435224364                   Callystor
155606403           

In [142]:
df_tweets_by_user = df_tweets.groupby(['user_id']).size().to_frame().reset_index()
df_most_favorited_users_with_info = df_most_favorited_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [143]:
df_most_favorited_users_with_info;

In [144]:
top_influencers_fav = df_most_favorited_users_with_info[['screen_name','description', 'followers_count', 'friends_count','favorite_count',0]]
top_influencers_rt = df_most_retweeted_users_with_info[['screen_name','description', 'followers_count', 'friends_count','retweet_count',0]]
top_influencers_fav.columns = ["Name","Description","Number of followers","Number of friends","Number of favorites","Number of tweets"]
top_influencers_rt.columns = ["Name","Description","Number of followers","Number of friends","Number of retweets","Number of tweets"]
top_influencers_fav = top_influencers_fav.reset_index().drop(columns = ["user_id"])
top_influencers_rt = top_influencers_rt.reset_index().drop(columns = ["user_id"]);
top_influencers_fav['Number of followers'] = top_influencers_fav['Number of followers'].astype(int)
top_influencers_fav['Number of friends'] = top_influencers_fav['Number of friends'].astype(int)
top_influencers_rt['Number of followers'] = top_influencers_rt['Number of followers'].astype(int)
top_influencers_rt['Number of friends'] = top_influencers_rt['Number of friends'].astype(int)

In [145]:
#add ratios
top_influencers_fav.loc[:,'Like Tweet Ratio'] = top_influencers_fav.loc[:,"Number of favorites"]/top_influencers_fav.loc[:,"Number of tweets"]
top_influencers_rt.loc[:,'Retweet Tweet Ratio'] = top_influencers_rt.loc[:,"Number of retweets"]/top_influencers_rt.loc[:,"Number of tweets"]

## Top hashtags for this input hashtags and by users

In [146]:
top_influencers_fav.style.set_properties(**{'text-align': 'left'});

In [147]:
df_tweets_hashtags = df_hashtags.set_index('tweet_id').join(df_tweets.set_index('tweet_id'))

In [148]:
df_tw_hash_user = df_tweets_hashtags.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))

In [149]:
#df_aggretated = df_tw_hash_user.groupby(['user_id','hashtag']).size().reset_index()
df_aggretated = df_tw_hash_user.groupby(['screen_name','hashtag']).size().reset_index()

In [150]:
df_aggretated.columns = ["Name", "#", "Used"]
df_aggretated;

In [151]:
#useful encapsulation, aint it?
def get_tweet_by_userid(user_id):
    df_tweets[df_tweets['user_id'] == 'user_id']

def get_tweet_by_username(name):
    df_tweets_user_info = df_tweets.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))
    df_tweets_user_info = df_tweets_user_info[df_tweets_user_info['screen_name'] == name]
    df_tweets_user_info = df_tweets_user_info[['tweet_id','text','created_at', 'favorite_count', 'retweet_count']]
    return df_tweets_user_info

def get_related_hashtags_by_username(name):
    d_name = df_aggretated[df_aggretated['Name'] == name][['#',"Used"]].sort_values("Used", ascending = False)
    return d_name
def get_user_info(name):
    return df_users[df_users['screen_name'] == name][['screen_name', 'description', 'followers_count' ]]

## propagation du bruit

In [163]:
top_influencers_fav_names = top_influencers_fav['Name'].values
top_influencers_rt_names = top_influencers_rt['Name'].values

In [164]:
df_most_retweeted_users_with_info = df_most_retweeted_users_with_info.reset_index()

In [175]:

df_retweet_users_top_account = df_retweet_users[df_retweet_users['original_user_screen_name'].isin(top_influencers_rt_names)]
rt_influencers  = df_retweet_users_top_account.groupby(['user_screen_name', 'original_user_screen_name']).size()
who_rt = rt_influencers.groupby('user_screen_name').size().sort_values(0, ascending = False).reset_index()
df_userid = df_users[['screen_name' , 'user_id']]
who_rt.columns = ['user_screen_name', 'count']
rt_influencers = rt_influencers.reset_index()

In [177]:
def who_rt_who(screenname):
    return rt_influencers[rt_influencers['user_screen_name'] == screenname]

# OUTPUTS


In [68]:
##OUTPUT: Stats de bases. Quantifier le ‘bruit’
print("Number of tweets scraped in the current folder: " + str(len(df_tweets.index)))
print("Number of retweets: " + str(int(df_tweets['retweet_count'].sum())))
print("Number of favorite: " + str(int(df_tweets['favorite_count'].sum())))


Number of tweets scraped in the current folder: 3149
Number of retweets: 26493
Number of favorite: 30536


In [94]:
##OUTPUT: Who are the most influential accounts?

print('top retweeted accounts:')
print(top_influencers_rt_names)
print('top favorited accounts:')
print(top_influencers_fav_names)

top retweeted accounts:
['GiletsJaunesFr' 'LeMediaTV' 'FranceNews24' 'Callystor' 'CitateurF'
 'AA36514431' 'MarchaiJ' 'tcabarrus' 'LorentzMathias' 'BBR4369'
 'LEDOUAISIEN' 'jmAlric11' 'ChLECHEVALIER' 'collCartonJaune' 'quatremer']
top favorited accounts:
['GiletsJaunesFr' 'LeMediaTV' 'tcabarrus' 'CitateurF' 'quatremer'
 'FranceNews24' 'LorentzMathias' 'Callystor' 'LEDOUAISIEN' 'MarchaiJ'
 'pe__ache' 'HRyssen' 'AA36514431' 'DidierMaisto' 'BBR4369']


In [85]:
##OUTPUT: Stats de base: Hashtags les plus frequents
df_aggretated.columns
df_total_hashtag = df_aggretated.groupby('#').size().sort_values(0,ascending = False).reset_index()
df_total_hashtag.columns = ['#', 'count']
df_total_hashtag.head(30)

,#,count
0,GiletJaune,1277
1,giletjaune,378
2,GiletsJaunes,270
3,Macron,127
4,giletsjaunes,95
5,Acte19,87
6,ActeXIX,84
7,Giletjaune,64
8,ActeXXI,63
9,ActeXX,59


In [96]:
##OUTPUT: INFO on the most favorited users
top_influencers_fav

,Name,Description,Number of followers,Number of friends,Number of favorites,Number of tweets,Like Tweet Ratio
0,GiletsJaunesFr,Mouvement citoyen apartisan pour une démocratie d'avenir. #GiletJaune #GiletsJaunes #RIC,28832,0,5655,52,108.750000
1,LeMediaTV,Soutenez un des seuls médias d'information indépendante et en accès libre pour le plus grand nombre : https://t.co/XC2TIKQzNc,57068,212,2085,1,2085.000000
2,tcabarrus,"Après 44 ans de journalisme ""objectif"" (carte de presse N°35183), je reprends ma liberté et j'abandonne une neutralité confortable. Ex chroniqueur sur Le Plus",8362,1063,1849,24,77.041667
3,CitateurF,"""Il est une bon de lire des livres de citations car les citations lorsqu'elles sont gravées dans la mémoire vous donnent de bonnes pensées."" \n\nWinston Churchill",236,22,1250,14,89.285714
4,quatremer,"Journaliste européen. Je bloque le vulgaire plus vite que mon ombre: comme le disait Lao Tseu, « tu as droit à une insulte, mais pas plus ».",158380,1154,1057,1,1057.000000
5,FranceNews24,France News 24 is France online news resource #Info #AlerteInfo #Breakingnews #Actualité #Media #Information #France #News,8680,3052,896,90,9.955556
6,LorentzMathias,,45,17,673,1,673.000000
7,Callystor,"Passionné d'#info, #politique, questions de #société. Amoureux de la #nature.🐝 Humaniste, mais sans illusion. #Insoumis, #fainéant, #rien et ami des #chats.",4843,3417,624,2,312.000000
8,LEDOUAISIEN,Rebel with many causes. https://t.co/ISNLNObWbX,7378,986,618,10,61.800000
9,MarchaiJ,Gaulois Provençal. La plume et le verbe faciles avec une franchise parfois dérangeante mais le respect est de rigueur. 100% Apolitique 🌈,324,439,554,8,69.250000


In [97]:
##OUTPUT: INFO on the most retweeted users
top_influencers_rt

,Name,Description,Number of followers,Number of friends,Number of retweets,Number of tweets,Retweet Tweet Ratio
0,GiletsJaunesFr,Mouvement citoyen apartisan pour une démocratie d'avenir. #GiletJaune #GiletsJaunes #RIC,28832,0,5196,52,99.923077
1,LeMediaTV,Soutenez un des seuls médias d'information indépendante et en accès libre pour le plus grand nombre : https://t.co/XC2TIKQzNc,57068,212,2464,1,2464.000000
2,FranceNews24,France News 24 is France online news resource #Info #AlerteInfo #Breakingnews #Actualité #Media #Information #France #News,8680,3052,1630,90,18.111111
3,Callystor,"Passionné d'#info, #politique, questions de #société. Amoureux de la #nature.🐝 Humaniste, mais sans illusion. #Insoumis, #fainéant, #rien et ami des #chats.",4843,3417,996,2,498.000000
4,CitateurF,"""Il est une bon de lire des livres de citations car les citations lorsqu'elles sont gravées dans la mémoire vous donnent de bonnes pensées."" \n\nWinston Churchill",236,22,968,14,69.142857
5,AA36514431,"🇫🇷 Il faut qu’on puisse se serrer les coudes et qu’on s’en prenne enfin au bon ennemi,ces 1% qui croient nous contrôler nous les 99%! ⚠️👁 suit moi je te suit",326,466,915,10,91.500000
6,MarchaiJ,Gaulois Provençal. La plume et le verbe faciles avec une franchise parfois dérangeante mais le respect est de rigueur. 100% Apolitique 🌈,324,439,892,8,111.500000
7,tcabarrus,"Après 44 ans de journalisme ""objectif"" (carte de presse N°35183), je reprends ma liberté et j'abandonne une neutralité confortable. Ex chroniqueur sur Le Plus",8362,1063,857,24,35.708333
8,LorentzMathias,,45,17,813,1,813.000000
9,BBR4369,Né au milieu des années 60 quand la France était la France 🇫🇷 \nGilets Jaunes pour faire Vivre ma Famille décemment de Mon Travail.\nVive l'Europe des Nations,3927,1946,596,8,74.500000


In [176]:
##OUTPUT: which users who retweet the most influential accounts?
who_rt.head(20)

,user_screen_name,count
0,lexi82600,11
1,Lenine13,10
2,MULLERsam1,10
3,alegria3007,10
4,cathyk69,10
5,DuPouvoirDachat,9
6,Mebanhmi1,9
7,sama_kenshiro,9
8,blabalade,9
9,Fdbfrancois,8


In [178]:
##output: who are the influencers retweeted by an user, e.g. lexi82600? 
who_rt_who('lexi82600')

,user_screen_name,original_user_screen_name,0
7955,lexi82600,AA36514431,1
7956,lexi82600,ChLECHEVALIER,1
7957,lexi82600,CitateurF,1
7958,lexi82600,FranceNews24,1
7959,lexi82600,GiletsJaunesFr,3
7960,lexi82600,LEDOUAISIEN,2
7961,lexi82600,LeMediaTV,1
7962,lexi82600,LorentzMathias,1
7963,lexi82600,MarchaiJ,1
7964,lexi82600,collCartonJaune,1


In [39]:
##OUTPUT: Get all tweets by username, eg GiletsJaunesFr
get_tweet_by_username('GiletsJaunesFr')

,tweet_id,text,created_at,favorite_count,retweet_count
user_id,,,,,
1028668294751039488,1108163259931205637,"Répartition des fruits du travail ! ""Rothschild and Co"" présente ici son ""Rapport annuel 2017"". Énormes bénéfices en pourcentage par rapport aux bénéfices déjà conséquent de 2016 ... L'hémorragie est permanente ! \n\n#GiletJaune #GiletsJaunes #Finance #banque",Wed Mar 20 00:28:04 +0000 2019,11.0,18.0
1028668294751039488,1108261628309573632,"Parler d'une manifestation à venir vous en rend responsable ! Y marcher vous en rend complice ! etc ... Choisir des têtes, les criminaliser pour l'exemple ... et ensuite ?\n#GiletJaune #GiletsJaunes #YellowVests #chalecosamarillos #coletesamarelos #Gelbwesten #Europeennes2019",Wed Mar 20 06:58:57 +0000 2019,82.0,53.0
1028668294751039488,1108296971641597952,Acte 18 : Micro trottoir auprès des Gilets Jaunes.\n\n#GiletJaune #GiletsJaunes #YellowVests #chalecosamarillos #coletesamarelos #Gelbwesten #Europeennes2019 #assembleenationale #senat #Europeennes2019 #France2 #BFM #RTFrance #LCI #Cnews #CNN #CBSNews,Wed Mar 20 09:19:23 +0000 2019,21.0,12.0
1028668294751039488,1108313029769347072,"C'est un tel privilège d'être ministre, un tel honneur de servir ce pays, que l'on peut faire un effort d'exemplarité !\n#GiletJaune #GiletsJaunes #YellowVests #Gelbwesten #Europeennes2019 #assembleenationale #senat #Europeennes2019 #France2 #BFM #RTFrance #LCI #Cnews",Wed Mar 20 10:23:12 +0000 2019,34.0,35.0
1028668294751039488,1108432646458232832,Quand la castagne devient l'unique projet politique ...\n#GiletJaune #GiletsJaunes #RIC #assembleenationale #senat\n#France2 #BFM #RTFrance #LCI #Cnews #CNN #CBSNews\n#YellowVests #chalecosamarillos #coletesamarelos #Gelbwesten #Europeennes2019 #ecologie #Climat,Wed Mar 20 18:18:31 +0000 2019,58.0,38.0
1028668294751039488,1108652526038147072,"Mutilés après mutilés, les médias sont au diapason, l'omerta sur les violences d'état est totale, la non-réponse politique aux frigos vides est criante, le peuple est alors criminalisé, la répression est totale.\n#GiletJaune #GiletsJaunes #YellowVests #Gelbwesten #Europeennes2019",Thu Mar 21 08:52:14 +0000 2019,743.0,606.0
1028668294751039488,1108825050071945216,"Pris par le temps, mais également par l'argent, les premiers tests de l' ""Exterminator GJ"" seront menés in situ, samedi prochain, boutique du souvenir, Arc de Triomphe.\n#GiletJaune #GiletsJaunes #Acte18 #Acte19\n#YellowVests #chalecosamarillos #Gelbwesten #armee #Europeennes2019",Thu Mar 21 20:17:47 +0000 2019,88.0,56.0
1028668294751039488,1109030876665602050,"Toute revendication sociale est étouffée dans l’œuf, grâce à l'action toujours incontrôlable des Black Blocs , un mouvement né en Allemagne dans les années 80, il y a plus de 35 ans ! Le 17 nov 2018 , il était simplement question de cela :\n#GiletJaune #GiletsJaunes #RIC",Fri Mar 22 09:55:40 +0000 2019,69.0,39.0
1028668294751039488,1109073160639598595,"Privatisons, privatisons ...\n#GiletJaune #GiletsJaunes #RIC #assembleenationale #senat\n#France2 #BFM #RTFrance #LCI #Cnews #CNN #CBSNews\n#YellowVests #chalecosamarillos #coletesamarelos #Gelbwesten #Europeennes2019 #ecologie #Climat #Privatisation",Fri Mar 22 12:43:41 +0000 2019,27.0,17.0


In [217]:
#output: top hashtags written to csv. you can also use function below
for name in top_influencers_rt_names:
    df_rel = get_related_hashtags_by_username(name)
    df_rel.to_csv('top_hashtags_by_'+name+'.csv',index=False)
    df_rel=[]

GiletsJaunesFr
FranceNews24
LorentzMathias
AA36514431
Callystor
LEDOUAISIEN
tcabarrus
quatremer
mojitobatbat
collCartonJaune
HRyssen
Maraine_Lapine
ion_positon
Circonscripti18
GaetanEscorbiac


In [38]:
##OUTPUT: What hashtags do influencers use? Call function below with one of the names
get_related_hashtags_by_username('GiletsJaunesFr')

,#,Used
2190,GiletJaune,50
2192,GiletsJaunes,49
2207,YellowVests,39
2216,chalecosamarillos,37
2186,Gelbewesten,32
2183,Europeennes2019,23
2199,RIC,20
2218,droitsdelhomme,20
2206,ViolencesPolicieres,17
2169,Amnestyinternational,17


# NLP

In [220]:
import spacy

In [221]:
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
import fr_core_news_sm
nlp = fr_core_news_sm.load()
#!python -m spacy download fr_core_web_lg

In [222]:
doc = nlp(df_tweets["text"][3])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [223]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [224]:
review = str(" ".join([i.lemma_ for i in doc]))

In [225]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [226]:
for i in nlp(review):
    print(i,"=>",i.pos_)

" => PUNCT
ce => DET
être => AUX
en => PRON
faire => VERB
ne => ADV
importer => VERB
quoi => PRON
, => PUNCT
que => SCONJ
le => DET
on => PRON
devenir => VERB
ne => ADV
importer => VERB
qui => PRON
" => PUNCT
on => PRON
accréditer => VERB
ce => DET
phrase => NOUN
à => ADP
@remigaillard => ADV
_ => ADP
   => SPACE
mais => CCONJ
désormais => ADV
ce => DET
être => AUX
un => DET
petit => ADJ
peu => ADV
le => DET
histoire => NOUN
de => ADP
@ccastaner => VERB
# => ADJ
manifestation => NOUN
# => ADJ
giletjaun => ADJ
# => ADJ
Castaner => PROPN


In [227]:
parser = French()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [228]:
tqdm.pandas()
df_tweets["processed_text"] = df_tweets["text"].progress_apply(spacy_tokenizer)

100%|██████████| 2002/2002 [00:04<00:00, 413.22it/s]


In [229]:
df_tweets.head(10)["processed_text"];

In [230]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words=stopwords, lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(df_tweets["processed_text"])

In [241]:
##How many topics do you want to find??
NUM_TOPICS = 3

In [242]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [243]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [244]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [245]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [246]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('dan', 108.92088327641676), ('faire', 79.4970493326865), ('prendre', 78.58320751174651), ('giletsjaunes', 67.89442328541232), ('mettre', 65.44960746443925), ('tre', 54.75800828266243), ('bon', 53.13962343077285), ('policier', 51.393623068165184), ('aller', 48.07522864835847), ('fran', 42.23916030885288)]
Topic 1:
[('france', 201.7985579281172), ('giletsjaunes', 164.84010604316458), ('plaire', 159.0722089414498), ('pouvoir', 154.55298414172606), ('faire', 141.2876673899798), ('macron', 134.4797669920172), ('dan', 89.06299769397921), ('vouloir', 81.43998247346931), ('aller', 77.86293571221583), ('ric', 65.85398649664629)]
Topic 2:
[('giletsjaunes', 453.8203145535721), ('acte', 453.49227649507776), ('actexix', 245.55913518165858), ('mars', 209.22646953553294), ('samedi', 154.64274740625223), ('nice', 135.9365285513162), ('macron', 125.57193090693772), ('jaune', 121.95282900012835), ('manifestation', 120.39932759765075), ('paris', 111.55050725584891)]


In [247]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('mars', 3.5922614901588923), ('samedi', 1.8866201376120633), ('actexix', 1.7115744401895998), ('acte', 1.6261130026342412), ('stopgiletsjaunes', 1.611407236693129), ('giletsjaunes', 1.3737159578656095), ('blackbloc', 1.3304897528756037), ('noborder', 1.2603536052198356), ('direct', 1.2143374017612385), ('antifa', 1.205529783100565)]
Topic 1:
[('giletsjaunes', 3.6820213248467812), ('acte', 2.79192374702542), ('actexix', 1.048551391292378), ('macron', 0.5650489328117727), ('gilets', 0.4731756340645095), ('actexx', 0.4716658151738041), ('manifestation', 0.4522571196151421), ('jaunes', 0.42877357530794546), ('yellowvests', 0.39346336696441514), ('castaner', 0.3832264772316482)]
Topic 2:
[('faire', 2.1444670656602227), ('france', 2.0335353708982264), ('macron', 1.7985092282362491), ('dan', 1.6893066476514293), ('plaire', 1.3053119429678095), ('pouvoir', 1.0577362306170561), ('aller', 0.8460891418652707), ('fran', 0.7157986816167251), ('politique', 0.6639552251283546), 

In [248]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('giletsjaunes', 0.5505396358520082), ('acte', 0.46845544463972444), ('mars', 0.31774558581878515), ('actexix', 0.2799902660729111), ('samedi', 0.18350764759423419), ('macron', 0.1420285710404683), ('stopgiletsjaunes', 0.13843691037047662), ('nice', 0.13276770955749298), ('france', 0.117677095764205), ('blackbloc', 0.10860019182721718)]
Topic 1:
[('mars', 0.44035786629551826), ('samedi', 0.2065202435989184), ('stopgiletsjaunes', 0.19717518766387707), ('blackbloc', 0.17434048313608305), ('noborder', 0.1677196752641014), ('antifa', 0.15998844301326348), ('direct', 0.15143650642631568), ('yellowvest', 0.13719106683507482), ('actexix', 0.13148763325831833), ('yellowjackets', 0.11361528411761693)]
Topic 2:
[('acte', 0.5161437998122204), ('giletsjaunes', 0.1428334796863568), ('gilets', 0.07473717513434407), ('actexx', 0.0707810540350995), ('jaunes', 0.0657696801019075), ('actexix', 0.0588415508788967), ('yellowvests', 0.04820093694614954), ('castaner', 0.0456037960501542

In [249]:
# Transforming an individual sentence
text = spacy_tokenizer("Les gilets jaune tous unis contre Macron. Tous dans la rue jusqu'à la démission")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.04573579 0.04770572 0.90655849]


In [250]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                 x           y  topics  cluster       Freq
topic                                                     
2      -632.045471 -384.325073       1        1  36.050339
1     -1209.665405  212.164124       2        1  35.094583
0     -1437.453247 -586.322510       3        1  28.855078, topic_info=     Category        Freq               Term       Total  loglift  logprob
6     Default  446.000000               acte  446.000000  30.0000  30.0000
8     Default  242.000000            actexix  242.000000  29.0000  29.0000
587   Default  200.000000               mars  200.000000  28.0000  28.0000
866   Default  149.000000             samedi  149.000000  27.0000  27.0000
657   Default  130.000000               nice  130.000000  26.0000  26.0000
757   Default  149.000000            pouvoir  149.000000  25.0000  25.0000
729   Default  163.000000             plaire  163.000000  24.0000  24.0000
427   Default  653.000000       giletsjaunes  653.000000  23.0000  23.0000
392   Default  255.000000             france  255.000000  22.0000  22.0000
494   Default  118.000000              jaune  118.000000  21.0000  21.0000
759   Default   77.000000            prendre   77.000000  20.0000  20.0000
699   Default  111.000000              paris  111.000000  19.0000  19.0000
574   Default  126.000000      manifestation  126.000000  18.0000  18.0000
10    Default   92.000000             actexx   92.000000  17.0000  17.0000
423   Default   88.000000              gilet   88.000000  16.0000  16.0000
611   Default   64.000000             mettre   64.000000  15.0000  15.0000
425   Default   78.000000             gilets   78.000000  14.0000  14.0000
495   Default   69.000000             jaunes   69.000000  13.0000  13.0000
280   Default   66.000000             direct   66.000000  12.0000  12.0000
957   Default   53.000000                tre   53.000000  11.0000  11.0000
1011  Default   82.000000            vouloir   82.000000  10.0000  10.0000
116   Default   53.000000                bon   53.000000   9.0000   9.0000
908   Default  101.000000   stopgiletsjaunes  101.000000   8.0000   8.0000
1021  Default   58.000000         yellowvest   58.000000   7.0000   7.0000
108   Default   61.000000          blackbloc   61.000000   6.0000   6.0000
661   Default   53.000000           noborder   53.000000   5.0000   5.0000
346   Default   59.000000        europeennes   59.000000   4.0000   4.0000
430   Default   51.000000  giletsjaunesparis   51.000000   3.0000   3.0000
53    Default   51.000000             antifa   51.000000   2.0000   2.0000
987   Default   39.000000                vid   39.000000   1.0000   1.0000
...       ...         ...                ...         ...      ...      ...
252    Topic3   18.463287              damer   19.199547   1.2038  -5.7134
152    Topic3   16.467290             causer   17.150352   1.2022  -5.8278
490    Topic3   17.016962        interpeller   17.742716   1.2011  -5.7950
297    Topic3   24.635913       dupontaignan   25.691914   1.2009  -5.4250
282    Topic3   14.437480          discourir   15.079712   1.1994  -5.9593
521    Topic3   16.659302            lection   17.425878   1.1979  -5.8162
360    Topic3   32.885875                fdo   34.969902   1.1814  -5.1361
44     Topic3   23.168245                amp   24.449775   1.1890  -5.4864
459    Topic3   19.571220              homme   20.565470   1.1933  -5.6551
134    Topic3   25.709721                but   27.605955   1.1717  -5.3823
344    Topic3   31.461838              europ   35.720943   1.1159  -5.1804
930    Topic3   22.159618              temps   23.974101   1.1642  -5.5309
1023   Topic3   36.269577            youtube   46.659895   0.9910  -5.0382
253    Topic3  105.416459                dan  241.593665   0.4135  -3.9712
742    Topic3   49.740083           policier   93.595050   0.6107  -4.7224
353    Topic3   76.939308              faire  247.254807   0.0755  -4.2861
31     Topic3   40.830886                ais   79.716077   0.5

In [254]:
df_tweets

,tweet_id,created_at,text,truncated,source,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user_id,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang
